In [6]:
import os
import time
import pickle
import re
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib widget
import numpy as np
import pandas as pd
pd.set_option('display.max_row', None)
pd.set_option('display.max_colwidth', None)
import ipywidgets as widgets
from IPython.display import Markdown, clear_output
plt.style.use("default")
folder = "/Working/Datasets/Dow_Data/"
from matplotlib.ticker import MaxNLocator
# plt.style.use("dark_background")
# plt.rcParams['figure.facecolor']=.07*np.ones(3)
# plt.rcParams['figure.edgecolor']=.07*np.ones(3)
# plt.rcParams['axes.facecolor']=.07*np.ones(3) 
# plt.rcParams['grid.linewidth']=.2
# plt.rcParams['axes.linewidth']=.5
plt.rcParams.update({'font.size': 14})

In [7]:
class GUI3:  
    def __init__(self):
        
        '''
        Constructor for the Graphical User Interface
        - The __init__ method lets the class initialize the object's attributes and serves no other purpose.
        - The attributes initialized are : -
            1. self.df : Dataframe containing inventory histories of 463 Plant-Material Pairs.
                         Each inventory history contains a minimum of 1000 datapoints and is tracked for over 3 years daily.
            2. self.dfPM : Dataframe containing a list of all Plant-Material pairs.
            3. self.dfCD : Dataframe containing a list of all class descriptions which represent different inventory behaviors.
            4. self.C_dict : Dictionary containing class memebership for each datapoint in self.df
            5. self.barge (300,000.00 * 1e-3) , self.rail_car (85,000.00 * 1e-3), self.truck (20,000.00 * 1e-3), self.barrel (300.00 * 1e-3) : Transportation mode constants.
        - The set_widgets(), set_canvas() and the printer() methods are also initialized here.
        '''
        
        self.df = pd.read_csv(os.path.join(folder,"Inv_hist2.csv"))
        self.dfPM = pd.DataFrame(columns=["Plant", "Material"], data=self.df[["Plant", "Material"]].drop_duplicates().values)
        self.dfCD = pd.read_csv(os.path.join(folder,"anomalyClassDescription.csv"),thousands=',', index_col=False)
        self.C_dict = pickle.load(open(os.path.join(folder,"pickles/D2_C_labels_464_listTuples.pickle"),"rb"))
        self.barge = 300_000.00 * 1e-3
        self.rail_car = 85_000.00 * 1e-3
        self.truck = 20_000.00 *1e-3
        self.barrel = 300.00 *1e-3
        self.set_widgets()
        self.set_canvas()
        self.printer()
    
    def set_widgets(self):
        
        '''
        set_widgets() method is used to initialize all the interactive properties of the graphical user interface.
        '''

        self.pm_w = widgets.Dropdown(options=[str(a.tolist()) for a in self.dfPM.values], description="Plant-Mat.",layout=widgets.Layout(width="200px"))
        self.show_count_w = widgets.IntText(disabled=True, value = len(self.pm_w.options),layout=widgets.Layout(width="90px"))
        self.Inv_w = widgets.Checkbox(description = "Inventory Position",value=True)
        self.listAll_w = widgets.Checkbox(value=True, description = "Show All", layout=widgets.Layout(width = '180px'))
        self.table2_w = widgets.SelectMultiple(disabled = True, options = [f"C{item_[0]}: {item_[1]}" for item_ in self.dfCD.values],layout=widgets.Layout(width='500px',height='200px'))
        self.transportModes_w = widgets.Checkbox(value=True, description = "Show Transport",indent=False, layout=widgets.Layout(width='175px', height='50px'))
        
        self.barge_w = widgets.Checkbox(value=False, description = "Barge",indent=False, layout=widgets.Layout(width='70px', height='50px'))
        self.rail_car_w = widgets.Checkbox(value=False, description = "Rail Car",indent=False, layout=widgets.Layout(width='70px', height='50px'))
        self.truck_w = widgets.Checkbox(value=False, description = "Truck",indent=False, layout=widgets.Layout(width='70px', height='50px'))
        self.barrel_w = widgets.Checkbox(value=False, description = "Barrel",indent=False, layout=widgets.Layout(width='70px', height='50px'))
        
        pm = eval(str(self.pm_w.value))
        if (pm[0],pm[1]) in self.C_dict:
            textBox_value  = self.C_dict[(pm[0],pm[1])]
        else:
            textBox_value = "()"
            
        self.classify_w = widgets.Text(value = str(textBox_value),description = "Description:", layout=widgets.Layout(width="500px") ) 
        self.submit_w = widgets.Button(description = "Store Description")
        self.canvas_w = widgets.Output()      
        self.displayLabels_w = widgets.Output()
        
        
        display(widgets.HBox([self.listAll_w,self.pm_w, self.show_count_w,self.barge_w,self.rail_car_w,self.truck_w,self.barrel_w])) 
        display(widgets.HBox([self.canvas_w, widgets.VBox([self.table2_w]) ])) 
        display(widgets.VBox([widgets.HBox([self.classify_w,  self.submit_w]), self.displayLabels_w]))
        self.submit_w.on_click(self.store_description)
        self.table2_w.observe(self.dropdown_eventhandler, names='value')
        self.listAll_w.observe(self.update_dropdown, 'value')

        for w in [self.pm_w]:
            w.observe(self.update_trace,names="value")
            w.observe(self.update_assignment,names="value")
            w.observe(self.update_count,names="value")
            
        for w in [self.barge_w,self.rail_car_w,self.truck_w,self.barrel_w]:
            w.observe(self.update_trace,names="value")
    
    def update_count(self,input_):
        '''
        update_count() method is used to display the number of options currently available in the Dropdown (which displays list of Plant-Material pairs)
        '''
        pairs_n = len(self.pm_w.options)
        self.show_count_w.value = pairs_n
        
    def update_dropdown(self,*args):
        
        '''
        The update_dropdown() method is used to allow the user to either choose between listing all Plant-Material pairs or to only display those of a particular class selected from the table.
        '''
        
        
        if self.listAll_w.value:
            self.table2_w.disabled = True
            self.pm_w.options = [str(a.tolist()) for a in self.dfPM.values]
        else:
            self.table2_w.disabled = False

        self.listAll_w.observe(self.update_dropdown, 'value')
  
    def dropdown_eventhandler(self,change):
        
        '''
        Based on the class selection in the table, this dropdown_eventhandler() method is used to display Plant-Material pairs of the selected class only.
        '''
        
        if not self.listAll_w.value:
                
            opts = []
            
            class_ =  int((re.findall(r'-?[1-8]|[0-9]', self.table2_w.value[0]))[0])#int((re.findall(r'[-+]?\d*\.\d+|\d+', self.table2_w.value[0]))[0]) #re.search(r'C(-?\d)', self.table2_w.value[0]) #int((re.findall(r'[-+]?\d*\.\d+|[-+]?\d+', "-1"))[0])#int((re.findall(r'[-+]?\d*\.\d+|\d+', self.table2_w.value[0]))[0])
            display(class_)
            for k, v in self.C_dict.items():
                for eT in v:
                    if class_ == eT[0]:
                        opts.append(str(k))
            self.pm_w.options = set(opts)

    def set_canvas(self):
        '''
        Creates a canvas for showing the data for the selected inventory history from the Dropdown.
        '''
        with self.canvas_w:

            with plt.ioff():
                num="p-m view"
                if plt.fignum_exists(num):plt.close(num)
                self.fig,self.ax=plt.subplots(num=num,figsize=(10,4))
                self.fig.canvas.header_visible=False
                self.fig.canvas.toolbar_position = 'bottom'
                self.fig.canvas.toolbar_visible=True
                self.fig.canvas.show()
                t,I,ylab = self.get_Inventory()
                
                try:
                    key = eval(self.pm_w.value)
                    classes = []
                    tModes = []
                    for (cl,si,ei) in self.C_dict[(key[0],key[1])]:

                        if cl in classes:
                            if cl != -1:
                                self.IP_trace, = self.ax.plot(np.arange(si,ei),I[si:ei],c=f'C{cl}',lw=1.5, marker = '.')
                            else:
                                self.IP_trace, = self.ax.plot(np.arange(si,ei),I[si:ei],c=f'C{cl+1}',lw=1.5, marker = '.')
                        else:
                            if cl != -1:
                                self.IP_trace, = self.ax.plot(np.arange(si,ei),I[si:ei],c=f'C{cl}',label=f'C{cl}',lw=1.5, marker = '.')
                            else:
                                self.IP_trace, = self.ax.plot(np.arange(si,ei),I[si:ei],c=f'C{cl+1}',label=f'C{cl}',lw=1.5, marker = '.')
                            classes.append(cl) 

                except:
                    display("Set Canvas Exception!")
                    self.IP_trace, = self.ax.plot(t,I)
   
                    
                if self.barge_w.value:# and max(I) > self.barge
                   
                    x = -60
                    y1 = min(I) if min(I) > 0 else 0 
                    y2 = y1 + self.barge 

                    self.ax.plot([x,x],[y1,y2],marker="_",color="lightgray",lw=0.5)
                    self.ax.text(x,y2,"  Barge",va = "bottom", ha='center', rotation = "vertical")

                if self.rail_car_w.value:# and max(I) > self.rail_car
            
                    x = -45
                    y1 = min(I) if min(I) > 0 else 0 
                    y2 = y1 + self.rail_car 

                    self.ax.plot([x,x],[y1,y2],marker="_",color="lightgray",lw=0.5)
                    self.ax.text(x,y2,"  Rail",va = "bottom", ha='center', rotation = "vertical")

                if self.truck_w.value:# and max(I) > self.truck
            
                    x = -30
                    y1 = min(I) if min(I) > 0 else 0 
                    y2 = y1 + self.truck 

                    self.ax.plot([x,x],[y1,y2],marker="_",color="lightgray",lw=0.5)
                    self.ax.text(x,y2,"  Truck",va = "bottom", ha='center', rotation = "vertical")
                    
                if self.barrel_w.value:# and max(I) > self.barrel
                    x = -15
                    y1 = min(I) if min(I) > 0 else 0 
                    y2 = y1 + self.barrel 

                    self.ax.plot([x,x],[y1,y2],marker="_",color="lightgray",lw=0.5)
                    self.ax.text(x,y2,"  Barrel",va = "bottom", ha='center', rotation = "vertical")
                               
                self.ax.set(xlabel="Time $t$ [day]", ylabel=ylab) 
                self.ax.legend(ncol = len(self.ax.lines))
                self.fig.tight_layout();
                
    def get_Inventory(self):
        '''
        The get_Inventory() method looks for the current value of the Dropdown and returns the data array associated to that inventory history
        '''
        pm = eval(str(self.pm_w.value))
        plant, material=pm[0], pm[1]
        if self.Inv_w.value == True:
            I = self.df[(self.df.Plant == plant) & (self.df.Material == material)][self.df.columns[-2:]].sum(axis=1).values*1e-3
         
            ylab=" $I_p$ [kg$\\times 10^3$] " # [kg$\\times 10^6$] Inventory position                   
        else:
            I = self.df[(self.df.Plant == plant) & (self.df.Material == material)][self.df.columns[3]].values*1e-3
            ylab="$I_b$ [kg$\\times 10^6$]"
        t = self.df[(self.df.Plant == plant) & (self.df.Material == material)].Day.values

        return(t,I,ylab)
    
    def update_trace(self,event):
        '''
        The update_trace method is used in updating the canvas by refreshing the plot and displaying the data associated with the current value of a Plant-Material pair.
        '''
   
        t,I,ylab = self.get_Inventory()
        with open('/Working/Datasets/Dow_Data/pickles/D2_C_labels_464_listTuples.pickle', 'rb') as io:
            try:
                cLabs_ = pickle.load(io)
                key = eval(self.pm_w.value)
                self.ax.clear()
                self.ax.clear()
                classes = []
                tModes = []

                for (cl,si,ei) in cLabs_[(key[0],key[1])]:
                    if cl in classes:
                        if cl != -1:
                            self.IP_trace, = self.ax.plot(np.arange(si,ei),I[si:ei],c=f'C{cl}',lw=1.5, marker = '.')       
                        else:
                            self.IP_trace, = self.ax.plot(np.arange(si,ei),I[si:ei],c=f'C{cl+1}',lw=1.5, marker = '.') 
                    else: 
                        if cl != -1:
                            self.IP_trace, = self.ax.plot(np.arange(si,ei),I[si:ei],c=f'C{cl}',label=f'C{cl}',lw=1.5, marker = '.')
                        else:
                            self.IP_trace, = self.ax.plot(np.arange(si,ei),I[si:ei],c=f'C{cl+1}',label=f'C{cl}',lw=1.5, marker = '.')
                        classes.append(cl)

                self.ax.legend()
              
            except:
                self.ax.clear()
                display("Update Trace Exception!")
                self.IP_trace, = self.ax.plot(t,I);
                display("Plot plain graph!")  
                
        if self.barge_w.value:# and max(I) > self.barge
                   

            x = -60
            y1 = min(I) if min(I) > 0 else 0 
            y2 = y1 + self.barge 

            self.ax.plot([x,x],[y1,y2],marker="_",color="lightgray",lw=0.5)
            self.ax.text(x,y2,"  Barge",va = "bottom", ha='center', rotation = "vertical")

        if self.rail_car_w.value:# and max(I) > self.rail_car


            x = -45
            y1 = min(I) if min(I) > 0 else 0 
            y2 = y1 + self.rail_car 

            self.ax.plot([x,x],[y1,y2],marker="_",color="lightgray",lw=0.5)
            self.ax.text(x,y2,"  Rail",va = "bottom", ha='center', rotation = "vertical")

        if self.truck_w.value:# and max(I) > self.truck


            x = -30
            y1 = min(I) if min(I) > 0 else 0 
            y2 = y1 + self.truck

            self.ax.plot([x,x],[y1,y2],marker="_",color="lightgray",lw=0.5)
            self.ax.text(x,y2,"  Truck",va = "bottom", ha='center', rotation = "vertical")
            
        if self.barrel_w.value:# and max(I) > self.barrel
            
            x = -15
            y1 = min(I) if min(I) > 0 else 0 
            y2 = y1 + self.barrel 

            self.ax.plot([x,x],[y1,y2],marker="_",color="lightgray",lw=0.5)
            self.ax.text(x,y2,"  Barrel",va = "bottom", ha='center', rotation = "vertical")
            
        self.ax.set(xlim=[t[0]-120, t[-1]+50],ylim=[I.min()-I.ptp()/20, I.max()+I.ptp()/10],ylabel=ylab, xlabel= "Time $t$ [day]")
        self.ax.legend(ncol = len(self.ax.lines))
        self.fig.tight_layout()
        self.fig.canvas.draw()
           
    def store_description(self,_input):
        
        '''
        store_description() method is utilized in storing short descriptions for the selected inventory history.
        - The entered text is stored in a dictionary with the key being the current Plant-Material selected in the Dropdown.
        - The value of that key is the input text entered.
        - This updated dictionary is then stored as a pickle file.
        '''

        inp_txt = self.classify_w.value
        pm = eval(self.pm_w.value)
        plant, material=pm[0], pm[1]

        with open('/Working/Datasets/Dow_Data/pickles/D2_C_labels_464_listTuples.pickle', 'rb+') as f:
            try:
                dictionary = pickle.load(f)
            except EOFError:
                dictionary = {}

            if isinstance(inp_txt, str):
                try:
                    inp_txt = inp_txt.replace("[", "").replace("]", "").replace("'","")
                except:
                    pass

                inp_txt = re.findall(r'-?\d+', inp_txt)
                inp_txt = list(map(int, inp_txt))
                inp_txt = [tuple(inp_txt[i:i+3]) for i in range(0, len(inp_txt), 3)]

            elif isinstance(inp_txt,list):
                inp_txt = self.convert2Tuple(inp_txt)

            if (plant,material) in dictionary:
                dictionary[(plant,material)] = inp_txt
            else:
                dictionary.setdefault((int(plant),int(material)),[]).append(inp_txt)

            f.seek(0)
            pickle.dump(dictionary, f)
        self.printer()
        self.update_trace(_input)
        self.dropdown_eventhandler(_input)
        return
    
    def update_assignment(self,event):
        
        '''
        The update_assignment() method obtains the recent text description stored for a particular inventory history and updates the description with any changes made to it.
        '''
        
        with open('/Working/Datasets/Dow_Data/pickles/D2_C_labels_464_listTuples.pickle', 'rb') as f:
                try:
                    dictionary = pickle.load(f)
                except EOFError:
                    dictionary = {}
        pm = eval(self.pm_w.value)
        
        if (pm[0],pm[1]) in dictionary:

            textBox_value  = dictionary[(pm[0],pm[1])]

        else:

            textBox_value = "()"
        
        self.classify_w.value = str(textBox_value)
        
    def convert2Tuple(self,list_values):
        
        '''
        The convert2Tuple method is used to convert the input string from the user on a particular Inventory History to a tuple.
        This tuple represents the indices (starting, ending) for a particular class membership assigned to an inventory history
        '''

        string_value = list_values[0]
        open_indices = [m.start() for m in re.finditer("\(", string_value)]
        end_indices = [m.start() for m in re.finditer("\)", string_value)]

        list_tuples = list()

        for idx_ in range(len(open_indices)):
            gather_ = string_value[open_indices[idx_]:end_indices[idx_]+1]
            commas_ = [m.start() for m in re.finditer(",", str(gather_))]
            classLabel = int(gather_[1:commas_[0]].strip())
            startIDX = int(gather_[commas_[0]+1:commas_[-1]].strip())
            endIDX = int(gather_[commas_[-1]+1:-1].strip())
            list_tuples.append((classLabel,startIDX,endIDX))
            
        return list_tuples
        
    def printer(self):
        '''
        The printer() method is used to display the latest entries to the dictionary of descriptions.
        '''
        self.displayLabels_w.clear_output()
        with self.displayLabels_w:
            with open('/Working/Datasets/Dow_Data/pickles/D2_C_labels_464_listTuples.pickle','rb') as printer:
                dK_dict = pickle.load(printer)
                last_ten = list(dK_dict.items())[-10:]
                for key, value in last_ten:
                    display(f"{key}: {value}")                 
g = GUI3()

In [ ]:
plant,material = 411,170# 431,3103 101,822 439,2486 54,459 80,348 43,774
I = g.df[(g.df.Plant == plant) & (g.df.Material == material)][g.df.columns[-2:]].sum(axis=1).values
len(I)

In [ ]:
100*1000

In [ ]:
# class GUIv2:
    
#     def __init__(self):
#         self.df = pd.read_csv(os.path.join(folder,"Inv_hist2.csv"))
#         self.dfPM = pd.DataFrame(columns=["Plant", "Material"], data=self.df[["Plant", "Material"]].drop_duplicates().values)
#         self.dfCD = pd.read_csv(os.path.join(folder,"weakClassDescriptions.csv"),thousands=',', index_col=False)
#         self.C_dict = pickle.load(open(os.path.join(folder,"cLabs.pickle"),"rb"))
#         self.set_widgets()
#         self.set_canvas()
#         self.printer()
    
#     def set_widgets(self):

#         self.pm_w = widgets.Dropdown(options=[str(a.tolist()) for a in self.dfPM.values], description="Plant-Mat.",layout=widgets.Layout(width="200px"))
#         self.show_count_w = widgets.IntText(disabled=True, value = len(self.pm_w.options),layout=widgets.Layout(width="90px"))
#         self.Inv_w = widgets.Checkbox(description = "Inventory Position",value=True)
#         self.listAll_w = widgets.Checkbox(value=True, description = "Show All")
#         self.table2_w = widgets.SelectMultiple(disabled = True, options = [f"C{item_[0]}: {item_[1]}" for item_ in self.dfCD.values],layout=widgets.Layout(height='190px',width="550px"))
#         with open('/Working/Datasets/Dow_Data/cLabs.pickle', 'rb') as f:
#                 try:
#                     dictionary = pickle.load(f)
#                 except EOFError:
#                     dictionary = {}
#         pm = eval(str(self.pm_w.value))
#         if (pm[0],pm[1]) in dictionary:
#             textBox_value  = dictionary[(pm[0],pm[1])]
#         else:
#             textBox_value = "()"
#         self.classify_w = widgets.Text(value = str(textBox_value),description = "Description:",layout=widgets.Layout(width="975px")) 
#         self.submit_w = widgets.Button(description = "Store Description")
#         self.canvas_w = widgets.Output()

        
#         self.displayLabels_w = widgets.Output()
#         display(widgets.HBox([widgets.VBox([widgets.HBox([self.listAll_w,self.pm_w,self.show_count_w]),self.canvas_w,widgets.HBox([self.classify_w,  self.submit_w ])]),self.table2_w]))
#         self.submit_w.on_click(self.store_description)
#         display(widgets.VBox([self.displayLabels_w]))
    
#         self.table2_w.observe(self.dropdown_eventhandler, names='value')
#         self.listAll_w.observe(self.update_dropdown, 'value')
#         # self.show_count_w.observe(self.update_count,'value')
        
#         for w in [self.pm_w]:
#             w.observe(self.update_trace,names="value")
#             w.observe(self.update_assignment,names="value")
#             w.observe(self.update_count,names="value")
            
#     def update_count(self,input_):
#         pairs_n = len(self.pm_w.options)
#         self.show_count_w.value = pairs_n
        
#     def update_dropdown(self,*args):
        
#             if self.listAll_w.value:
#                 self.table2_w.disabled = True
#                 self.pm_w.options = [str(a.tolist()) for a in self.dfPM.values]
#             else:
#                 self.table2_w.disabled = False

#             self.listAll_w.observe(self.update_dropdown, 'value')
  
#     def dropdown_eventhandler(self,change):

#         opts = []
#         class_ = int((re.findall(r'[-+]?\d*\.\d+|\d+', self.table2_w.value[0]))[0])
#         display(class_)
#         for k, v in self.C_dict.items():
#             for eT in v:
#                 if class_ == eT[0]:
#                     opts.append(str(k))
#         self.pm_w.options = set(opts)
    
#     def set_canvas(self):
#         """
#         create a canvas for showing the data
#         """
#         with self.canvas_w:

#             with plt.ioff():
#                 num="p-m view"
#                 if plt.fignum_exists(num):plt.close(num)
#                 self.fig,self.ax=plt.subplots(num=num,figsize=(8,3),sharey='col')
#                 self.fig.canvas.header_visible=False
#                 self.fig.canvas.toolbar_position = 'bottom'
#                 self.fig.canvas.toolbar_visible=True
#                 self.fig.canvas.show()
#                 self.ax.ticklabel_format(useOffset=False, style='plain')
                
#                 # self.ax.set_yscale('log')
#                 t,I,ylab = self.get_Inventory()
           
#                 with open('/Working/Datasets/Dow_Data/class_Labels.pickle', 'rb') as io:
#                     try:

#                         cLabs_ = pickle.load(io)
#                         key = eval(self.pm_w.value)
#                         classes = []
#                         transport_mode = []
#                         for (cl,si,ei) in cLabs_[(key[0],key[1])]:
#                             if cl in classes:
#                                 self.IP_trace, = self.ax.plot(np.arange(si,ei),I[si:ei],c=f'C{cl}')
#                                 diff = max(np.diff(I[si:ei]))
#                                 print(diff)
#                             else:
#                                 self.IP_trace, = self.ax.plot(np.arange(si,ei),I[si:ei],c=f'C{cl}',label=f'C{cl}')
#                                 classes.append(cl)     
                
#                     except:
#                         display("Set Canvas Exception!")
#                         self.IP_trace, = self.ax.plot(t,I)
                               
#                 self.ax.set(xlabel="Time $t$ [day]", ylabel=ylab) #yscale="log",
#                 # self.ax.axhline(color="red", ls =":")
#                 self.ax.ticklabel_format(useOffset=False, style='plain')
#                 self.ax.legend()
#                 self.fig.tight_layout();
                
#     def get_Inventory(self):
#         pm = eval(str(self.pm_w.value))
#         plant, material=pm[0], pm[1]
#         if self.Inv_w.value == True:
#             I = self.df[(self.df.Plant == plant) & (self.df.Material == material)][self.df.columns[-2:]].sum(axis=1).values #*1e-6
#             ylab="Inventory Position $I_p$ [kg] " # [kg$\\times 10^6$] Inventory position                   
#         else:
#             I = self.df[(self.df.Plant == plant) & (self.df.Material == material)][self.df.columns[3]].values #*1e-6
#             ylab="$I_b$ [kg$\\times 10^6$]"
#         t = self.df[(self.df.Plant == plant) & (self.df.Material == material)].Day.values

#         return(t,I,ylab)
    
#     def update_trace(self,event):
        
#         # self.set_canvas()

#         t,I,ylab = self.get_Inventory()
#         # display(np.diff(I), len(np.diff(I)), max(np.diff(I)))
#         with open('/Working/Datasets/Dow_Data/cLabs.pickle', 'rb') as io:
#                     try:

#                         cLabs_ = pickle.load(io)
#                         key = eval(self.pm_w.value)
#                         self.ax.clear()
#                         classes = []
#                         for (cl,si,ei) in cLabs_[(key[0],key[1])]:
#                             # display("Inside for loop of tuples")
#                             if cl in classes:
#                                 # display("CL is in CLASSES!")
#                                 self.IP_trace, = self.ax.plot(np.arange(si,ei),I[si:ei],c=f'C{cl}') 
#                                 diff = max(np.diff(I[si:ei]))
#                                 # if diff < 85000:
#                                 #     self.ax.plot(si,)
#                             else:
                                
#                                 self.IP_trace, = self.ax.plot(np.arange(si,ei),I[si:ei],c=f'C{cl}',label=f'C{cl}')
#                                 classes.append(cl)
#                                 # display("We are in ELSE!", classes)
                            
#                         self.ax.legend()
#                         self.ax.ticklabel_format(useOffset=False, style='plain')
#                         # display("Still in TRY!")
                        
#                     except:
#                         self.ax.clear()
#                         display("Update Trace Exception!")
#                         self.IP_trace, = self.ax.plot(t,I);
#                         display("Plot plain graph!")

                        
        
#         self.ax.set(xlim=[t[0], t[-1]],ylim=[I.min()-I.ptp()/20, I.max()+I.ptp()/20],ylabel=ylab, xlabel= "Time $t$ [day]") #yscale ="log",
#         self.ax.legend()
#         self.fig.tight_layout()
#         self.fig.canvas.draw()
           
#     def store_description(self,_input):

#             inp_txt = self.classify_w.value
#             pm = eval(self.pm_w.value)
#             plant, material=pm[0], pm[1]
#             # self.classify_w.value = "()"

#             with open('/Working/Datasets/Dow_Data/cLabs.pickle', 'rb+') as f:
#                 try:
#                     dictionary = pickle.load(f)
#                 except EOFError:
#                     dictionary = {}
                    
#                 if isinstance(inp_txt, str):
#                     try:
#                         inp_txt = inp_txt.replace("[", "").replace("]", "").replace("'","")
#                     except:
#                         pass
                    
#                     inp_txt = re.findall(r'\d+', inp_txt)
#                     inp_txt = list(map(int, inp_txt))
#                     inp_txt = [tuple(inp_txt[i:i+3]) for i in range(0, len(inp_txt), 3)]
                    
#                 elif isinstance(inp_txt,list):
#                     inp_txt = self.convert2Tuple(inp_txt)
                        
#                 if (plant,material) in dictionary:
#                     dictionary[(plant,material)] = inp_txt
#                 else:
#                     dictionary.setdefault((int(plant),int(material)),[]).append(inp_txt)
                    
#                 f.seek(0)
#                 pickle.dump(dictionary, f)
#             self.printer()
#             self.update_trace(_input)
#             self.dropdown_eventhandler(_input)
#             return
    
#     def update_assignment(self,event):
        
#         with open('/Working/Datasets/Dow_Data/cLabs.pickle', 'rb') as f:
#                 try:
#                     dictionary = pickle.load(f)
#                 except EOFError:
#                     dictionary = {}
#         pm = eval(self.pm_w.value)
        
#         if (pm[0],pm[1]) in dictionary:

#             textBox_value  = dictionary[(pm[0],pm[1])]

#         else:

#             textBox_value = "()"
        
#         self.classify_w.value = str(textBox_value)
        
#     def convert2Tuple(self,list_values):

#         string_value = list_values[0]
#         open_indices = [m.start() for m in re.finditer("\(", string_value)]
#         end_indices = [m.start() for m in re.finditer("\)", string_value)]

#         list_tuples = list()

#         for idx_ in range(len(open_indices)):
#             gather_ = string_value[open_indices[idx_]:end_indices[idx_]+1]
#             commas_ = [m.start() for m in re.finditer(",", str(gather_))]
#             classLabel = int(gather_[1:commas_[0]].strip())
#             startIDX = int(gather_[commas_[0]+1:commas_[-1]].strip())
#             endIDX = int(gather_[commas_[-1]+1:-1].strip())
#             list_tuples.append((classLabel,startIDX,endIDX))
            
#         return list_tuples
        
#     def printer(self):
#         self.displayLabels_w.clear_output()
#         with self.displayLabels_w:
#             with open('/Working/Datasets/Dow_Data/cLabs.pickle','rb') as printer:
#                 dK_dict = pickle.load(printer)
#                 last_ten = list(dK_dict.items())[-10:]
#                 for key, value in last_ten:
#                     print(f"{key}: {value}")                 
# g = GUIv2()

In [ ]:
# class GUI:
    
#     def __init__(self):
#         self.df = pd.read_csv(os.path.join(folder,"Inv_hist2.csv"))
#         self.dfPM = pd.DataFrame(columns=["Plant", "Material"], data=self.df[["Plant", "Material"]].drop_duplicates().values)
#         self.dfCD = pd.read_csv(os.path.join(folder,"weakClassDescriptions.csv"),thousands=',', index_col=False)
#         self.C_dict = pickle.load(open(os.path.join(folder,"class_Labels.pickle"),"rb"))
#         self.set_widgets()
#         self.set_canvas()
#         self.set_Table()
#         self.display_Table()
#         self.printer()
    
#     def set_widgets(self):
#         self.class_w = widgets.Dropdown(disabled = True, description = "By Class",options = self.dfCD[self.dfCD.columns[0]].values,layout=widgets.Layout(width="200px"))
#         self.desc_w = widgets.Text(value='', placeholder='Selected value will appear here', disabled=True,layout=widgets.Layout(width="375px"))
#         self.pm_w = widgets.Dropdown(options=[str(a.tolist()) for a in self.dfPM.values], description="Plant-Mat.",layout=widgets.Layout(width="200px"))
#         self.Inv_w = widgets.Checkbox(description = "Inventory Position",value=True)
#         self.listAll_w = widgets.Checkbox(value=True, description = "Show All")
#         with open('/Working/Datasets/Dow_Data/class_Labels.pickle', 'rb') as f:
#                 try:
#                     dictionary = pickle.load(f)
#                 except EOFError:
#                     dictionary = {}
#         pm = eval(str(self.pm_w.value))
        
#         if (pm[0],pm[1]) in dictionary:
#             textBox_value  = dictionary[(pm[0],pm[1])]
#         else:
#             textBox_value = "()"

#         self.classify_w = widgets.Text(value = str(textBox_value), 
#                                        description = "Assign Class(es):",
#                                        layout=widgets.Layout(width="975px"))
        
#         self.submit_w = widgets.Button(description = "Store Description")
    
#         self.canvas_w = widgets.Output()
#         self.table_w = widgets.Output()
#         self.displayLabels_w = widgets.Output()
#         display(widgets.HBox([widgets.VBox([widgets.HBox([self.listAll_w,self.pm_w,self.class_w,self.desc_w]),self.canvas_w, 
#                                             widgets.HBox([self.classify_w,  self.submit_w ])]),
#                               self.table_w]))
#         self.submit_w.on_click(self.store_description)
#         display(widgets.VBox([self.displayLabels_w]))
        
#         self.class_w.observe(self.dropdown_eventhandler, names='value')
#         self.listAll_w.observe(self.update_dropdown, 'value')
        
#         for w in [self.pm_w, self.Inv_w]:
#             w.observe(self.update_trace,names="value")
#             w.observe(self.update_assignment,names="value")
        
#     def update_dropdown(self,*args):
#             if self.listAll_w.value:
#                 self.class_w.disabled = True
#                 self.pm_w.options = [str(a.tolist()) for a in self.dfPM.values]
#             else:
#                 self.class_w.disabled = False
#                 self.desc_w.value = str(self.dfCD.at[eval(self.class_w.value),"Class Description"])
#                 self.dropdown_eventhandler()
  
#     def dropdown_eventhandler(self,change):
        
#         self.desc_w.value = str(self.dfCD.at[self.class_w.value,"Class Description"])
#         opts = []
#         class_ = int(self.class_w.value)
#         for k, v in self.C_dict.items():
#             for eT in v:
#                 if class_ == eT[0]:

#                     opts.append(str(k))
#         self.pm_w.options = set(opts)
    
#     def set_canvas(self):
#         """
#         create a canvas for showing the data
#         """
#         with self.canvas_w:

#             with plt.ioff():
#                 num="p-m view"
#                 if plt.fignum_exists(num):plt.close(num)
#                 self.fig,self.ax=plt.subplots(num=num,figsize=(8,3))
#                 self.fig.canvas.header_visible=False
#                 self.fig.canvas.toolbar_visible=False
#                 self.fig.canvas.show()
#                 t,I,ylab = self.get_Inventory()
#                 with open('/Working/Datasets/Dow_Data/class_Labels.pickle', 'rb') as io:
#                     try:

#                         cLabs_ = pickle.load(io)
#                         key = eval(self.pm_w.value)
#                         classes = []
#                         for (cl,si,ei) in cLabs_[(key[0],key[1])]:
#                             if cl in classes:
#                                 self.IP_trace, = self.ax.plot(np.arange(si,ei),I[si:ei],c=f'C{cl}')
#                             else:
#                                 self.IP_trace, = self.ax.plot(np.arange(si,ei),I[si:ei],c=f'C{cl}',label=f'C{cl}')
#                                 classes.append(cl)     
                
#                     except:
#                         display("Set Canvas Exception!")
#                         self.IP_trace, = self.ax.plot(t,I)
                               
#                 self.ax.set(xlabel="Time $t$ [day]", ylabel=ylab)
#                 # self.ax.axhline(color="red", ls =":")
#                 self.ax.legend()
#                 self.fig.tight_layout()
                
#     def get_Inventory(self):
#         pm = eval(str(self.pm_w.value))
#         plant, material=pm[0], pm[1]
#         if self.Inv_w.value == True:
#             I = self.df[(self.df.Plant == plant) & (self.df.Material == material)][self.df.columns[-2:]].sum(axis=1).values*1e-6
#             ylab="Inventory position $I_p$ [kg$\\times 10^6$]"
#         else:
#             I = self.df[(self.df.Plant == plant) & (self.df.Material == material)][self.df.columns[3]].values*1e-6
#             ylab="$I_b$ [kg$\\times 10^6$]"
#         t = self.df[(self.df.Plant == plant) & (self.df.Material == material)].Day.values

#         return(t,I,ylab)
    
#     def update_trace(self,event):
        
#         # self.set_canvas()

#         t,I,ylab = self.get_Inventory()
#         with open('/Working/Datasets/Dow_Data/class_Labels.pickle', 'rb') as io:
#                     try:

#                         cLabs_ = pickle.load(io)
#                         key = eval(self.pm_w.value)
#                         self.ax.clear()
#                         classes = []
#                         for (cl,si,ei) in cLabs_[(key[0],key[1])]:
#                             display("Inside for loop of tuples")
#                             if cl in classes:
#                                 self.IP_trace, = self.ax.plot(np.arange(si,ei),I[si:ei],c=f'C{cl}')
#                             else:
#                                 self.IP_trace, = self.ax.plot(np.arange(si,ei),I[si:ei],c=f'C{cl}',label=f'C{cl}')
#                                 classes.append(cl)
                            
#                         self.ax.legend()
                        
#                     except:
#                         self.ax.clear()
#                         display("Update Trace Exception!")
#                         self.IP_trace, = self.ax.plot(t,I)
#                         display("Plot plain graph!")
#                         # self.ax.legend()
#                         # self.IP_trace.set_xdata(t)
#                         # self.IP_trace.set_ydata(I)
                       
                        
        
#         self.ax.set(xlim=[t[0], t[-1]],ylim=[I.min()-I.ptp()/20, I.max()+I.ptp()/20],ylabel=ylab,xlabel= "Time $t$ [day]")
#         self.ax.legend()
#         self.fig.tight_layout()
#         self.fig.canvas.draw()
    
#     def set_Table(self):
#         with self.table_w:
#             self.classL_w = widgets.IntText(description = "Label:",layout=widgets.Layout(width="150px"))
#             self.classD_w = widgets.Text(description = "Description:",layout=widgets.Layout(width="250px"))
#             self.submitCD_w = widgets.Button(description = "Update Table",layout=widgets.Layout(width="100px"))      
#             self.submitCD_w.on_click(self.update_Table)
#             self.deleteC_w = widgets.Text(description='Label:',layout=widgets.Layout(width="200px"))
#             self.submitDelete_w = widgets.Button(description = "Delete class!", layout=widgets.Layout(width="100px"))
#             self.submitDelete_w.on_click(self.delete_Table)
            
#     def update_Table(self,_input):
#         print(self.classL_w.value, type(self.classL_w.value), type(self.dfCD['Class ID'][0]))
#         self.dfCD.loc[len(self.dfCD['Class ID']),['Class ID','Class Description']]  = np.int64(self.classL_w.value), self.classD_w.value
#         self.classL_w.value = len(self.dfCD['Class ID'])
#         self.classD_w.value = ""  
#         self.display_Table()
#         return

#     def delete_Table(self,_input):
#         if int(self.deleteC_w.value) in self.dfCD['Class ID'].values:
#             self.dfCD = self.dfCD[self.dfCD['Class ID']!=int(self.deleteC_w.value)]
#             self.dfCD = self.dfCD.reset_index(drop=True)
#             self.dfCD['Class ID'] = range(len(self.dfCD))
#         self.classL_w.value = len(self.dfCD['Class ID'])
#         self.deleteC_w.value = ""
#         self.display_Table()    

#     def display_Table(self):
#         self.table_w.clear_output()
#         with self.table_w:
#             display(self.dfCD[["Class ID","Class Description"]])
#             display(widgets.VBox([widgets.HBox([self.classL_w,self.classD_w,self.submitCD_w]), widgets.HBox([self.deleteC_w,self.submitDelete_w])]))
        
#     def store_description(self,_input):

#             inp_txt = self.classify_w.value
#             pm = eval(self.pm_w.value)
#             plant, material=pm[0], pm[1]
#             # self.classify_w.value = "()"

#             with open('/Working/Datasets/Dow_Data/class_Labels.pickle', 'rb+') as f:
#                 try:
#                     dictionary = pickle.load(f)
#                 except EOFError:
#                     dictionary = {}
                    
#                 if isinstance(inp_txt, str):
#                     try:
#                         inp_txt = inp_txt.replace("[", "").replace("]", "").replace("'","")
#                     except:
#                         pass
                    
#                     inp_txt = re.findall(r'\d+', inp_txt)
#                     inp_txt = list(map(int, inp_txt))
#                     inp_txt = [tuple(inp_txt[i:i+3]) for i in range(0, len(inp_txt), 3)]
                    
#                 elif isinstance(inp_txt,list):
#                     inp_txt = self.convert2Tuple(inp_txt)
                        
#                 if (plant,material) in dictionary:
#                     dictionary[(plant,material)] = inp_txt
#                 else:
#                     dictionary.setdefault((int(plant),int(material)),[]).append(inp_txt)
                    
#                 f.seek(0)
#                 pickle.dump(dictionary, f)
#             self.printer()
#             self.update_trace(_input)
#             self.dropdown_eventhandler(_input)
#             return
    
#     def update_assignment(self,event):
        
#         with open('/Working/Datasets/Dow_Data/class_Labels.pickle', 'rb') as f:
#                 try:
#                     dictionary = pickle.load(f)
#                 except EOFError:
#                     dictionary = {}
#         pm = eval(self.pm_w.value)
        
#         if (pm[0],pm[1]) in dictionary:

#             textBox_value  = dictionary[(pm[0],pm[1])]

#         else:

#             textBox_value = "()"
        
#         self.classify_w.value = str(textBox_value)
        
#     def convert2Tuple(self,list_values):

#         string_value = list_values[0]
#         open_indices = [m.start() for m in re.finditer("\(", string_value)]
#         end_indices = [m.start() for m in re.finditer("\)", string_value)]

#         list_tuples = list()

#         for idx_ in range(len(open_indices)):
#             gather_ = string_value[open_indices[idx_]:end_indices[idx_]+1]
#             commas_ = [m.start() for m in re.finditer(",", str(gather_))]
#             classLabel = int(gather_[1:commas_[0]].strip())
#             startIDX = int(gather_[commas_[0]+1:commas_[-1]].strip())
#             endIDX = int(gather_[commas_[-1]+1:-1].strip())
#             list_tuples.append((classLabel,startIDX,endIDX))
            
#         return list_tuples
        
#     def printer(self):
#         self.displayLabels_w.clear_output()
#         with self.displayLabels_w:
#             with open('/Working/Datasets/Dow_Data/class_Labels.pickle','rb') as printer:
#                 dK_dict = pickle.load(printer)
#                 last_ten = list(dK_dict.items())[-10:]
#                 for key, value in last_ten:
#                     # print(f"{key}: {value}")
# gui = GUI()

## Examining Steady Production (C0) :- Count = 7

### Notes:-
- Saw-Teeth, reverse build up and then a quick dip in levels


## Examining Turnaround (C1) :- Count = 6

### Notes :-
- Double hump , coupled with ususally C2 or C0 and C5

## Examining Make to Stock (C2) :- Count = 176

### Notes :-
- Inventory doesn't touch 0
- Replenishments are periodic, deterministic


## Examining Make to Order (C3) :- Count = 269

### Notes:-
- Discrete spikes at the time of order.
- Without orders the inventory stays at 0


## Examining Constrained Supply (C4) :- Count = 13

### Notes :-

    
    

|SL| PM Pair | C4 Recognized in | Number of Replenishments | Closest Transport Mode | Other classes? | Notes |
|------:|--------------:|:-------------------------:|----------:|----------:|-----------:|-----:|
| 1 | (76,2689) | 185 days |  0 replenishments | no production | C2, C3 | Sum of all values in this interval will be less than sum in any other interval |
| 2 | (157,1112) | 263 days | 6 replenishments (85K) | Rail Cars  | C2,C3 | Inventory doesn't go above 100K. Sum of transations in this period will be less than sum of transactions in any other period of the same length |
| 3 | (76,2975) | 216 days | 2 replenishments (13K) | Trucks |C2| NA|
| 4 | (417,1719) | 356 days | 5 replenishments (40K) | Trucks | C2| NA|
| 5 | (240,2279) | 170 days | 5 replenishments (13K and 50K) | Trucks? | C2| NA|
| 6 | (127,720) | 216 days | 6 replenishments (13K) | Trucks | C2 | NA|
| 7 | (76,1039) | 213 days | 0 replenishments | no productions | C2 | NA|
|8| (461,3089)|267 days| 5 replenishmets (7K,3K(x3), 16K) | Less than truck | C2 | How is this C4| 
| 9| (76,1986) | 389 days | 2 replenishments (20K) | Truck | C2 | NA|
|10| (362,2279)|195 days| 14 replenishments (100K) | Barge | C2 | replenishments in C2 was 490K, but in C4 it is at 100K |
|11| (507,1092)|200 days | 1 replenishment (14K) | Truck| C2 |C2 replenishments are at 60K but C4 replenishments are at 15K|
|12| (210,3219) | 35 days | 1 replenishment (10K) | Half a truck | C2 | C2 replenishments are Rail Cars of 80K but C4 is Truck at 10K |
|13| (95,1060) | 482 days | 20 replenishments (10K,20K,40K) | trucks | C2 | C2 replenishments are 100K|

## Examining Downstream Turnaround + Consumption Paused (C5) :- Count = 4

### Notes :-
- (67,986) - recognized in 200 days, rapid reduction in production, inventory levels go to almost 0
- (67,2784) - recognized in 360 days
- (18,404) - 45,55,45 days are recognized as C5. In between these sections is a gap of 130,175 days. This is with C0 - so if its C0 then we see this
- (289,3075) - not sure!
- C5 is with C0,C2,C1,C9

## Examining Product Cycle (C6) :- Count = 3

### Notes :-
- 3 pairs are present
- All are recognized with a long window 
- Exhibiting saw-teeth
- Used all of data available for each PM pair


## Examining Order Entry Error (C7) :- Count = 4

### Notes :-
- Difficult

## Examining Similar to 0 with poor entries (C8) :- Count = 1


### Notes :
- Only 1 pair (125,1307), one with the order entry error
- Not sure how to interpret this

## Examining Dormant/Obsolete/No Demand (C9) :- Count = 107



### Notes :
- 107 pairs contain C9
- Almost always with C2, but also found with C0, C3 and C5
- Smallest  Windows :-

|SL| PM Pair | C9 Recognized in | 
|------:|--------------:|--------------:|
| 1 | (236,168) | 60 days |
| 2 | (318,2275) | 90 days |
|3| (210,2783) | 102,122,136 days|
|4|(210,2816) | 126,210 days|
|5|(268,3011) | 140,144,158,167 days|
|6| (275,986) | 177 days|
|7|(250,1810) | 180 days|

## Examining Project Orders (C10) :- Count  = 9


### Notes :  
- Almost always coupled/found with C3

|SL| PM Pair | C10 Recognized in | Increments in | Closest Transport Mode |
|------:|--------------:|:-------------------------:|----------:|----------:|
|1.| (426,887) | 164 days |  75K-85K | Rail Car |
|2.| (54,1189)  | 140 days | 80K | Rail Car|
|3.| (443,3201) | 90 days|  40K | Truck(s)|
|4.| (254,729) | 160 days|  33K and 86K |maybe used both Rail Cars and Trucks|
|5.| (109,792) | 190 days| 500 and 800  | not sure!|
|6.| (385,2113) | 50 days| 2500  | not sure!|
|7.| (421,3179) | 81 days| 400K  | Deep Sea Vessel|
|8.| (385,2677) | 37 days| 20K |  Truck(s) |
|9.| (67,2558) | 238 days| maybe 2 project orders, 60K  | maybe 3 Trucks or 1 Rail Car|

